<a href="https://colab.research.google.com/github/jcarlos03100890/CicloID2/blob/main/Relational_database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Check to see if we're running in Colab

In [ ]:
try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

if IN_COLAB:
  print("We're running Colab")

We're running Colab


# Switch current directory to a directory on colab where we will be working.

In [ ]:
import os
current_path = os.getcwd()
print("Current Working Directory: ",current_path)

# Switch to the directory on the Google Drive that you want to use
drive_root = "/content/Reto1_Ciclo2_Data"

# Create drive_root if it doesn't exist
create_drive_root = True
if create_drive_root:
  print("\nColab: making sure ", drive_root, " exists.")
  os.makedirs(drive_root, exist_ok=True)

  # Change to the directory
print("\nColab: Changing directory to ", drive_root)
%cd $drive_root

Current Working Directory:  /content

Colab: making sure  /content/Reto1_Ciclo2_Data  exists.

Colab: Changing directory to  /content/Reto1_Ciclo2_Data
/content/Reto1_Ciclo2_Data


In [ ]:
# Verify we're in the correct working directory
%pwd

'/content/Reto1_Ciclo2_Data'

## Fill the working directory from our Git repo !

In [ ]:
# Let's load the working directory from our Git repo !
repo_url = "https://github.com/jcarlos03100890/Reto1_data.git"

# Clone the repo unless it is already there
repo_root = os.path.join(drive_root,"Reto1_data")

# Clone or refresh from the repo depending on whether we've cloned before
if not os.path.exists( repo_root ):
  print("Cloning from Git url: ", repo_url)
  ! git clone $repo_url
else:
  print("Refreshing from Git url: ", repo_url)
  %cd $repo_root
  ! git pull


print("\nChange working directory to repo root")
%cd $repo_root

Refreshing from Git url:  https://github.com/jcarlos03100890/Reto1_data.git
/content/Reto1_Ciclo2_Data/Reto1_data
Already up to date.

Change working directory to repo root
/content/Reto1_Ciclo2_Data/Reto1_data


In [ ]:
# Verify which files are in
%ls

'Bike Store Data'/   Online-Shopping-Dataset.csv   README.md   tweets_extraction.json


# Relationa Database Analysis

## Tweets Analyisis

### Testing Database Connection

1. Verifying that Mysql Connector is already installed

In [ ]:
#"!pip install mysql.connector
!pip install mysql-connector-python

2. Testing Database Connection

In [ ]:
import mysql.connector
from mysql.connector import Error

try:
    connection = mysql.connector.connect(host='sql5.freemysqlhosting.net',
                                         database='sql5669281',
                                         user='sql5669281',
                                         password='jkbymIn9G5',
                                         port="3306")
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

Connected to MySQL Server version  5.5.62-0ubuntu0.14.04.1
You're connected to database:  ('sql5669281',)
MySQL connection is closed


### Read the file and storage within the data base

1. Read the JSON File.

In [ ]:
import pandas as pd

#Step 1: Read CSV File into a Pandas DataFrame
#nRowsRead = 1000
#df = pd.read_json("tweets_extraction.json", lines=True, chunksize = 10000)
df = pd.read_json("tweets_extraction.json")

#df = df.drop([0, 100])
# Limit the rows in order to send to mysql
df = df.drop(labels=range(99, 290605), axis=0)

#Cleaning the hashtag field, since it is a list and myssql doesn't support list
def clean(x):
    x = ','.join(x)
    return str(x)

df['hashtags'] = df['hashtags'].apply(clean)

# To display the rows
print("\n\nTo display Rows\n")
print(df.shape)
#df.dtypes
# To display the top 5 rows
print("\n\nTo display the top 5 rows\n")
print(df.head(10))





To display Rows

(100, 7)


To display the top 5 rows

                  id                                              texto  \
0  12116719578733158  #BrechaAnuario2019 | Con cañón y coronel El vo...   
1  12116547084876964  #BrechaAnuario2019 | Las PASO en Argentina "Em...   
2  12116529643744992  #BrechaAnuario2019 | Bolsonaro contra la selva...   
3  12110290516301292  #BrechaAnuario2019 | Aquí, los temas que apare...   
4  12107119614801142  #BrechaAnuario2019 | El mejor dibujante de his...   
5   1210681764626092  #BrechaAnuario2019 | Rock’n Rimbaud En noviemb...   
6   1210666663915344  #BrechaAnuario2019 | Primavera latinoamericana...   
7  12106515668160266  #BrechaAnuario2019 | A veces, gana la vida El ...   
8   1210637722655834  #BrechaAnuario2019 | Omnipresencia La estrateg...   
9  12106276560576676  #BrechaAnuario2019 | Se buscan narradoras, viv...   

           usuario                                        hashtags  \
0  SemanarioBrecha  BrechaAnuario2019,Relectura

In [ ]:
#Checking up that we have the proper library to send the data to mysql
!pip install sqlalchemy

3. Create a table within the database from df

In [ ]:
from sqlalchemy import create_engine
# Step 2: MySQL Database Connection Parametersd
# MySQL connection parameters
db_user = 'sql5669281'
db_password = 'jkbymIn9G5'
db_host = 'sql5.freemysqlhosting.net'
db_port = "3306"
# Database name to storage data
db_name = 'sql5669281'

# Step 3: Create a SQLAlchemy Engine to Connect to MySQL
engine = create_engine(f"mysql+mysqlconnector://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

# Step 4: Write DataFrame to MySQL
table_name = 'tweets'
df.to_sql(table_name, con=engine, index=False, if_exists='replace')  # Change 'replace' to 'append' if needed

# Step 5: Close the Database Connection
engine.dispose()

### Validating that the data has been set properly

In [ ]:
import pandas as pd
connection = mysql.connector.connect(host='sql5.freemysqlhosting.net',
                                         database='sql5669281',
                                         user='sql5669281',
                                         password='jkbymIn9G5',
                                         port="3306")

# Create a MySQL cursor
cursor = connection.cursor()

# Check if the database already exists
cursor.execute("SELECT * FROM tweets")

table_rows = cursor.fetchall()

cursor.close()
connection.close()

my_data = pd.DataFrame(table_rows)

# To display the rows
print("\n\nTo display Rows\n")
print(my_data.shape)
#df.dtypes
# To display the top 5 rows
print("\n\nTo display the top 10 rows\n")
print(my_data.head(10))



To display Rows

(100, 7)


To display the top 10 rows

                   0                                                  1  \
0  12116719578733158  #BrechaAnuario2019 | Con cañón y coronel El vo...   
1  12116547084876964  #BrechaAnuario2019 | Las PASO en Argentina "Em...   
2  12116529643744992  #BrechaAnuario2019 | Bolsonaro contra la selva...   
3  12110290516301292  #BrechaAnuario2019 | Aquí, los temas que apare...   
4  12107119614801142  #BrechaAnuario2019 | El mejor dibujante de his...   
5   1210681764626092  #BrechaAnuario2019 | Rock’n Rimbaud En noviemb...   
6   1210666663915344  #BrechaAnuario2019 | Primavera latinoamericana...   
7  12106515668160266  #BrechaAnuario2019 | A veces, gana la vida El ...   
8   1210637722655834  #BrechaAnuario2019 | Omnipresencia La estrateg...   
9  12106276560576676  #BrechaAnuario2019 | Se buscan narradoras, viv...   

                 2                                               3  \
0  SemanarioBrecha  BrechaAnuario2019,Relectur

In [ ]:
my_data.dtypes

0     int64
1    object
2    object
3    object
4    object
5     int64
6     int64
dtype: object

## Bike Store Relational Database

### Read the files

In [ ]:
import pandas as pd

# Directory containing CSV files
current_path = os.getcwd()
directory_path = current_path + '/Bike Store Data/'
print("Current Working Directory: ",directory_path)
# List all files in the directory
files = os.listdir(directory_path)
print("Files: ",files)
# Filter only CSV files
csv_files = [file for file in files if file.endswith('.csv')]

# Create an empty list to store DataFrames
dataframes_list = []
files = []
# Loop through each CSV file and read into DataFrame
for csv_file in csv_files:
  file_path = os.path.join(directory_path, csv_file)
  df = pd.read_csv(file_path)

  # Append the current DataFrame to the list
  dataframes_list.append(df)
  # Get the file names to set the Table Names
  files.append(os.path.splitext(csv_file)[0])

# Now, dataframes_list contains all DataFrames from the CSV files in the directory
print("Table Names: ",files)


Current Working Directory:  /content/Reto1_Ciclo2_Data/Reto1_data/Bike Store Data/
Files:  ['stocks.csv', 'staffs.csv', 'categories.csv', 'products.csv', 'orders.csv', 'brands.csv', 'stores.csv', 'order_items.csv', 'customers.csv']
Files:  ['stocks', 'staffs', 'categories', 'products', 'orders', 'brands', 'stores', 'order_items', 'customers']


### Validate the data read from the files

In [ ]:
# For example, print the shape of each DataFrame, table name, datatypes and the 5 fisrt rows
for i, df in enumerate(dataframes_list):
    print(f"\n\nDataFrame {i+1} Shape: {df.shape}")
    print(f"\n\nFile {i+1} Name: {files[i]}")
    print("\n\nDatatypes:\n",df.dtypes)
    print("\n\nFirst 5 rows:\n",df.head(5))



DataFrame 1 Shape: (939, 3)


File 1 Name: stocks


Datatypes:
 store_id      int64
product_id    int64
quantity      int64
dtype: object


First 5 rows:
    store_id  product_id  quantity
0         1           1        27
1         1           2         5
2         1           3         6
3         1           4        23
4         1           5        22


DataFrame 2 Shape: (10, 8)


File 2 Name: staffs


Datatypes:
 staff_id        int64
first_name     object
last_name      object
email          object
phone          object
active          int64
store_id        int64
manager_id    float64
dtype: object


First 5 rows:
    staff_id first_name last_name                       email           phone  \
0         1    Fabiola   Jackson  fabiola.jackson@bikes.shop  (831) 555-5554   
1         2     Mireya  Copeland  mireya.copeland@bikes.shop  (831) 555-5555   
2         3      Genna   Serrano    genna.serrano@bikes.shop  (831) 555-5556   
3         4     Virgie   Wiggins   virgie.wiggi

### Send the data to database

In [ ]:
# Once we have the data frame with the data, we will storage them within database
from sqlalchemy import create_engine

# Create a function to send the data to the database
def insert_table(table_name,df):
  # Step 1: MySQL Database Connection Parametersd
  # MySQL connection parameters
  db_user = 'sql5669281'
  db_password = 'jkbymIn9G5'
  db_host = 'sql5.freemysqlhosting.net'
  db_port = "3306"
  # Database name to storage data
  db_name = 'sql5669281'

  # Step 2: Create a SQLAlchemy Engine to Connect to MySQL
  engine = create_engine(f"mysql+mysqlconnector://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

  # Step 3: Write DataFrame to MySQL
  df.to_sql(table_name, con=engine, index=False, if_exists='replace')  # Change 'replace' to 'append' if needed

  # Step 5: Close the Database Connection
  engine.dispose()

# The loop to send the dataframe elements to insert_table function
for i, df in enumerate(dataframes_list):
    insert_table(files[i],df)



### Checking up the data storage in the database

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text

# Step 1: MySQL Database Connection Parametersd
# MySQL connection parameters
db_user = 'sql5669281'
db_password = 'jkbymIn9G5'
db_host = 'sql5.freemysqlhosting.net'
db_port = "3306"
# Database name to storage data
db_name = 'sql5669281'

# Step 2: Create a SQLAlchemy Engine to Connect to MySQL
engine = create_engine(f"mysql+mysqlconnector://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

# Step 3: Define the query
# Run SQL query and load results into a DataFrame
query = """
SHOW TABLES;
"""
# Step 4: Execute the query and get the data in a dataframe
df = pd.DataFrame(engine.connect().execute(text(query)))
df

,Tables_in_sql5669281
0,brands
1,categories
2,customers
3,order_items
4,orders
5,products
6,staffs
7,stocks
8,stores
9,tweets


In [ ]:
# Step 3: Define the query
# Run SQL query and load results into a DataFrame
query = """
SELECT
    *
FROM
    categories;
"""
# Step 4: Execute the query and get the data in a dataframe
df = pd.DataFrame(engine.connect().execute(text(query)))
df

,category_id,category_name
0,1,Children Bicycles
1,2,Comfort Bicycles
2,3,Cruisers Bicycles
3,4,Cyclocross Bicycles
4,5,Electric Bikes
5,6,Mountain Bikes
6,7,Road Bikes


In [ ]:
# Step 3: Define the query
# Run SQL query and load results into a DataFrame
query = """
SELECT
    first_name,
    last_name
FROM
    customers
ORDER BY
    last_name;
"""
# Step 4: Execute the query and get the data in a dataframe
df = pd.DataFrame(engine.connect().execute(text(query)))
df

,first_name,last_name
0,Jamika,Acevedo
1,Penny,Acevedo
2,Ester,Acevedo
3,Shery,Acosta
4,Bettyann,Acosta
...,...,...
1440,Edda,Young
1441,Jasmin,Young
1442,Jayme,Zamora
1443,Alexandria,Zamora


### WHERE, ORDER BY

In [ ]:
# Step 3: Define the query
# Run SQL query and load results into a DataFrame
query = """
SELECT
    first_name,
    last_name
FROM
    customers
WHERE
    last_name = 'Acevedo'
ORDER BY
    first_name;
"""
# Step 4: Execute the query and get the data in a dataframe
df = pd.DataFrame(engine.connect().execute(text(query)))
df

,first_name,last_name
0,Ester,Acevedo
1,Jamika,Acevedo
2,Penny,Acevedo


### WHERE, AND, OR

In [ ]:
# Step 3: Define the query
# Run SQL query and load results into a DataFrame
query = """
SELECT
    first_name,
    last_name
FROM
    customers
WHERE
    last_name = 'Acevedo'
AND
    first_name = 'Ester'
OR
    first_name = 'Jamika';
"""
# Step 4: Execute the query and get the data in a dataframe
df = pd.DataFrame(engine.connect().execute(text(query)))
df

,first_name,last_name
0,Jamika,Acevedo
1,Jamika,Blanchard
2,Ester,Acevedo


### Aggregate Functions


In [ ]:
# Step 3: Define the query
# Run SQL query and load results into a DataFrame
query = """
SELECT
    *,
    MAX(list_price)
FROM
    order_items;
"""

# Step 4: Execute the query and get the data in a dataframe
df = pd.DataFrame(engine.connect().execute(text(query)))
df

,order_id,item_id,product_id,quantity,list_price,discount,MAX(list_price)
0,1364,2,155,2,11999.99,0.1,11999.99


Los nombres automáticos dados a las columnas como resultado de una función no siempre son claros, así que usemos "AS" para cambiar el nombre de las columnas en la consulta de pedidos totales.

In [ ]:
# Step 3: Define the query
# Run SQL query and load results into a DataFrame
query = """
SELECT
    SUM(quantity) AS total_items_ordered
FROM
    order_items;
"""

# Step 4: Execute the query and get the data in a dataframe
df = pd.DataFrame(engine.connect().execute(text(query)))
df

,total_items_ordered
0,7078


### Arithmetic Operators
**Arithmetic operators** are used to perform mathematical calculations on numbers.


In [ ]:
# Step 3: Define the query
# Run SQL query and load results into a DataFrame
query = """
SELECT
    AVG(list_price * discount) AS avg_discount_usd
FROM
    order_items;
"""

# Step 4: Execute the query and get the data in a dataframe
df = pd.DataFrame(engine.connect().execute(text(query)))
df

,avg_discount_usd
0,126.698149


### Comparison Operators
Comparison operators are used to compare values ​​and return a true or false result. One of these, `=`, we already learned when using `WHERE`.

In [ ]:
# Step 3: Define the query
# Run SQL query and load results into a DataFrame
query = """
SELECT
    ROUND(AVG(list_price * (1 - discount)), 2) AS avg_saleprice_usd
FROM
    order_items
WHERE
    list_price >= 1000;
"""

# Step 4: Execute the query and get the data in a dataframe
df = pd.DataFrame(engine.connect().execute(text(query)))
df

,avg_saleprice_usd
0,2542.02


### GROUP BY & HAVING
`GROUP BY` divides the rows returned from a `SELECT` statement into one or more groups. What makes this so powerful is that you can apply aggregate functions like `SUM()` to each group. This is often used to aggregate data by unique identifier, time period, category, etc.

In [ ]:
# Step 3: Define the query
# Run SQL query and load results into a DataFrame
query = """
SELECT
    order_id,
    ROUND(SUM(quantity * list_price * (1 - discount)), 2) final_sale_price
FROM
    order_items
GROUP BY
    order_id;
"""

# Step 4: Execute the query and get the data in a dataframe
df = pd.DataFrame(engine.connect().execute(text(query)))
df

,order_id,final_sale_price
0,1,10231.05
1,2,1697.97
2,3,1519.98
3,4,1349.98
4,5,3900.06
...,...,...
1610,1611,8963.96
1611,1612,3781.13
1612,1613,5257.97
1613,1614,6104.04


`HAVING` works similarly to `WHERE`, except that it is used to filter aggregate values ​​output by `SUM()`, etc.

Let's use this in the same query as above to only include final sale prices over $20,000.

Positional references such as 1 can be used in place of column names in the `ORDER BY` and `GROUP BY` clauses to improve readability and ease of writing. The order is determined by the order of the columns in the `SELECT` clause.

In [ ]:
# Step 3: Define the query
# Run SQL query and load results into a DataFrame
query = """
SELECT
    order_id,
    ROUND(SUM(quantity * list_price * (1 - discount)), 2) final_sale_price
FROM
    order_items
GROUP BY
    1
HAVING
    final_sale_price > 20000
ORDER BY
    2 DESC;
"""

# Step 4: Execute the query and get the data in a dataframe
df = pd.DataFrame(engine.connect().execute(text(query)))
df

,order_id,final_sale_price
0,1541,29147.03
1,937,27050.72
2,1506,25574.96
3,1482,25365.43
4,1364,24890.62
5,930,24607.03
6,1348,20648.95
7,1334,20509.43
8,973,20177.75


### INNER JOIN
In a relational database, data is typically distributed across multiple tables, as is the case with our sample database. To select complete data, you often need to query and combine data from multiple tables. We do this by using various table joins.

For joins to be possible, the tables in question must both share a column such as order_id. The query will then find the rows in both tables where the values are equal and join them.

An `INNER JOIN` keeps only the rows that have a match in both tables.

![PostgreSQL-Join-Inner-Join.png](attachment:7cbec9f9-2839-4071-ad0c-f30611d49758.png)

Let's join the orders table with the customers table as an example.

In [ ]:
# Step 3: Define the query
# Run SQL query and load results into a DataFrame
query = """
SELECT
    *
FROM
    orders
INNER JOIN
    customers
ON
    orders.customer_id = customers.customer_id
LIMIT 5;
"""
# Step 4: Execute the query and get the data in a dataframe
df = pd.DataFrame(engine.connect().execute(text(query)))
df

,order_id,customer_id,order_status,order_date,required_date,shipped_date,store_id,staff_id,customer_id,first_name,last_name,phone,email,street,city,state,zip_code
0,1,259,4,2016-01-01,2016-01-03,2016-01-03,1,2,259,Johnathan,Velazquez,None,johnathan.velazquez@hotmail.com,9680 E. Somerset Street,Pleasanton,CA,94566
1,2,1212,4,2016-01-01,2016-01-04,2016-01-03,2,6,1212,Jaqueline,Cummings,None,jaqueline.cummings@hotmail.com,478 Wrangler St.,Huntington Station,NY,11746
2,3,523,4,2016-01-02,2016-01-05,2016-01-03,2,7,523,Joshua,Robertson,None,joshua.robertson@gmail.com,81 Campfire Court,Patchogue,NY,11772
3,4,175,4,2016-01-03,2016-01-04,2016-01-05,1,3,175,Nova,Hess,None,nova.hess@msn.com,773 South Lafayette St.,Duarte,CA,91010
4,5,1324,4,2016-01-03,2016-01-06,2016-01-06,2,6,1324,Arla,Ellis,None,arla.ellis@yahoo.com,127 Crescent Ave.,Utica,NY,13501


If a customer_id was present in the orders table but not in the customer table, that row would be excluded. The opposite is true as well, if a customer_id was present in the customers table but not in the orders table, that customer's information would be excluded from the final output.

### LEFT JOIN
A `LEFT JOIN` works similarly to an `INNER JOIN`, except that it keeps all the records in the first table, even if there is no match. In the case of no match the data for the columns in the right table are filled with NULL values.

![PostgreSQL-Join-Left-Join.png](attachment:a5f38c81-93f4-4c3d-a3cb-9c021143a8cf.png)

In [ ]:
# Step 3: Define the query
# Run SQL query and load results into a DataFrame
query = """
SELECT
    *
FROM
    orders
LEFT JOIN
    customers
ON
    orders.customer_id = customers.customer_id
LIMIT 5;
"""
# Step 4: Execute the query and get the data in a dataframe
df = pd.DataFrame(engine.connect().execute(text(query)))
df

,order_id,customer_id,order_status,order_date,required_date,shipped_date,store_id,staff_id,customer_id,first_name,last_name,phone,email,street,city,state,zip_code
0,1,259,4,2016-01-01,2016-01-03,2016-01-03,1,2,259,Johnathan,Velazquez,None,johnathan.velazquez@hotmail.com,9680 E. Somerset Street,Pleasanton,CA,94566
1,2,1212,4,2016-01-01,2016-01-04,2016-01-03,2,6,1212,Jaqueline,Cummings,None,jaqueline.cummings@hotmail.com,478 Wrangler St.,Huntington Station,NY,11746
2,3,523,4,2016-01-02,2016-01-05,2016-01-03,2,7,523,Joshua,Robertson,None,joshua.robertson@gmail.com,81 Campfire Court,Patchogue,NY,11772
3,4,175,4,2016-01-03,2016-01-04,2016-01-05,1,3,175,Nova,Hess,None,nova.hess@msn.com,773 South Lafayette St.,Duarte,CA,91010
4,5,1324,4,2016-01-03,2016-01-06,2016-01-06,2,6,1324,Arla,Ellis,None,arla.ellis@yahoo.com,127 Crescent Ave.,Utica,NY,13501


Now, unlike in the `INNER JOIN`, if a customer_id was present in the orders table but not in the customers table, the rows in orders containing that id would be included in the final output.

### FULL OUTER JOIN
A `FULL OUTER JOIN` includes all the rows from both tables, even if there is no match. Like the `LEFT JOIN`, when there is no match for a row the data from the other table is filled with NULL values for that row.

![PostgreSQL-Join-Full-Outer-Join.png](attachment:c315e1ca-9747-42e1-aa49-75666954a9f7.png)

In [ ]:
# Step 3: Define the query
# Run SQL query and load results into a DataFrame
query = """
SELECT
    *
FROM
    orders
FULL JOIN
    customers
ON
    orders.customer_id = customers.customer_id
LIMIT 5;
"""

# Step 4: Execute the query and get the data in a dataframe
df = pd.DataFrame(engine.connect().execute(text(query)))
df

,order_id,customer_id,order_status,order_date,required_date,shipped_date,store_id,staff_id,customer_id,first_name,last_name,phone,email,street,city,state,zip_code
0,1,259,4,2016-01-01,2016-01-03,2016-01-03,1,2,259,Johnathan,Velazquez,None,johnathan.velazquez@hotmail.com,9680 E. Somerset Street,Pleasanton,CA,94566
1,2,1212,4,2016-01-01,2016-01-04,2016-01-03,2,6,1212,Jaqueline,Cummings,None,jaqueline.cummings@hotmail.com,478 Wrangler St.,Huntington Station,NY,11746
2,3,523,4,2016-01-02,2016-01-05,2016-01-03,2,7,523,Joshua,Robertson,None,joshua.robertson@gmail.com,81 Campfire Court,Patchogue,NY,11772
3,4,175,4,2016-01-03,2016-01-04,2016-01-05,1,3,175,Nova,Hess,None,nova.hess@msn.com,773 South Lafayette St.,Duarte,CA,91010
4,5,1324,4,2016-01-03,2016-01-06,2016-01-06,2,6,1324,Arla,Ellis,None,arla.ellis@yahoo.com,127 Crescent Ave.,Utica,NY,13501


Again, here all the rows from both orders and customers would be included, even where there was no match.

### SELF JOIN
A **SELF JOIN** is a regular join that joins a table to itself. Typically, these are used to query hierarchical data or to compare rows within the same table.

To create a SELF JOIN, you use the same table twice with different table aliases.

One example of where you can apply a SELF JOIN is pulling the most recent and second most recent purchase dates for customers with at least 2 orders. A more complex version of this could be used to calculate the average days between transactions, which is often used to segment customers in e-commerce.

In [ ]:
# Step 3: Define the query
# Run SQL query and load results into a DataFrame
query = """
SELECT
    t1.customer_id,
    MAX(t1.order_date) most_recent_order,
    MAX(t2.order_date) second_most_recent_order
FROM
    orders t1
INNER JOIN
    orders t2
ON
    t1.customer_id = t2.customer_id
AND
    t1.order_date > t2.order_date
GROUP BY
    1;
"""

# Step 4: Execute the query and get the data in a dataframe
df = pd.DataFrame(engine.connect().execute(text(query)))
df

,customer_id,most_recent_order,second_most_recent_order
0,1,2018-11-18,2018-04-18
1,2,2018-04-09,2017-08-21
2,3,2018-10-21,2018-04-06
3,4,2018-04-18,2017-11-21
4,5,2018-04-17,2016-11-24
...,...,...,...
126,231,2018-04-26,2017-12-14
127,233,2018-04-13,2017-06-03
128,237,2018-04-08,2016-04-11
129,239,2018-04-29,2017-08-09


### UNION
With joins, we have been combining data horizontally, but with `UNION` we combine it vertically. `UNION` combines the results of 2 or more `SELECT` statements.

The conditions for this are that the number, order, and data types of the columns are the same. `UNION` removes duplicate rows. In the case you wish to keep these `UNION ALL` should be used instead.

As an example, let's pretend that our store has an extremely large product catalog, and keeps their products in different tables separated by year. We want to get the products for 2016 and 2019, so we can use `UNION` to combine the results.

In [ ]:
# Step 3: Define the query
# Run SQL query and load results into a DataFrame
query = """
SELECT
    product_id,
    product_name,
    model_year,
    list_price
FROM
    products
WHERE
    model_year = 2016
UNION
SELECT
    product_id,
    product_name,
    model_year,
    list_price
FROM
    products
WHERE
    model_year = 2019;
"""
# Step 4: Execute the query and get the data in a dataframe
df = pd.DataFrame(engine.connect().execute(text(query)))
df

,product_id,product_name,model_year,list_price
0,1,Trek 820 - 2016,2016,379.99
1,2,Ritchey Timberwolf Frameset - 2016,2016,749.99
2,3,Surly Wednesday Frameset - 2016,2016,999.99
3,4,Trek Fuel EX 8 29 - 2016,2016,2899.99
4,5,Heller Shagamaw Frame - 2016,2016,1320.99
5,6,Surly Ice Cream Truck Frameset - 2016,2016,469.99
6,7,Trek Slash 8 27.5 - 2016,2016,3999.99
7,8,Trek Remedy 29 Carbon Frameset - 2016,2016,1799.99
8,9,Trek Conduit+ - 2016,2016,2999.99
9,10,Surly Straggler - 2016,2016,1549.00
